# Background

**calibration:**  
Agreement between observed outcomes and predictions

**Discrimination:**  
Ability of model predictions to discriminate between those with and those without the outcome

# set environment

In [41]:
library(tidyverse)
library(pROC)
library(IRdisplay)

In [18]:
source("./data/beetle.R")
beetle

dose,n,y,n_y
1.6907,59,6,53
1.7242,60,13,47
1.7552,62,18,44
1.7842,56,28,28
1.8113,63,52,11
1.8369,59,53,6
1.8610,62,61,1
1.8839,60,60,0


https://discuss.analyticsvidhya.com/t/how-to-get-the-percentage-concordant-and-discordant-values-for-a-logistic-regression-model-in-r/1458

In [53]:
concordance <- function(model){
    
    # Get all actual observations and their fitted values into a frame
    fitted <- data.frame(cbind(model$y, model$fitted.values))
    colnames(fitted)<-c('respvar','score')
    
    # Subset only ones
    ones  <- fitted[fitted[,1] == 1,]
    # Subset only zeros
    zeros <- fitted[fitted[,1] == 0,]
    
    #print(ones)
    #print(zeros)
    # Initialise all the values
    pairs_tested <- 0
    conc         <- 0
    disc         <- 0
    ties         <- 0
      
    # Get the values in a for-loop
    for(i in 1:nrow(ones)) {
          
        for(j in 1:nrow(zeros)) {
            pairs_tested<-pairs_tested + 1
            
            if        (ones[i, 2] >  zeros[j, 2]) { 
                conc <- conc + 1 
            } else if (ones[i, 2] == zeros[j, 2]){ 
                ties <- ties + 1 
            } else { 
                disc <- disc + 1 
            } # end if-else
        } # end inner for
    } # end outer for
    
    # Calculate concordance, discordance and ties
    concordance <- conc / pairs_tested
    discordance <- disc / pairs_tested
    ties_perc   <- ties / pairs_tested
    n           <- nrow(fitted)
    
    # index
    # http://support.sas.com/kb/45/767.html
    # http://support.sas.com/documentation/cdl/en/statug/66103/HTML/default/viewer.htm#statug_surveylogistic_details64.htm
    return(
        list(
            "Concordance" = concordance,
            "Discordance" = discordance,
            "Tied"        = ties_perc,
            "Pairs"       = pairs_tested,
            
            ### Somers' D
            "Somers D"    = (concordance - discordance),
            
            ### Goodman-Kruskal Gamma
            "gamma"       = (concordance - discordance) / (concordance + discordance),
            
            ### Kendall's Tau-a
            "tau-a"       = (conc - disc) / (0.5 * n * (n - 1)),
            
            ### c-statistics
            "c"           = auc(model$y, model$fitted.values)
        ) # end list
    ) # end return
} # end func

In [54]:
fit_logit    = glm(cbind(y, n - y) ~ dose, data = beetle,    family = binomial(link = "logit"))
fit_logit_n1 = glm(cbind(y, n - y) ~ dose, data = beetle_n1, family = binomial(link = "logit"))

In [62]:
res = concordance(fit_logit_n1)
display(res)

$Concordance
[1] 0.8698318

$Discordance
[1] 0.06766142

$Tied
[1] 0.06250678

$Pairs
[1] 55290

$`Somers D`
[1] 0.8021704

$gamma
[1] 0.8556546

$`tau-a`
[1] 0.3841996

$c
Area under the curve: 0.9011

In [63]:
res = concordance(fit_logit)
display(res)

ERROR: Error in if (ones[i, 2] > zeros[j, 2]) {: missing value where TRUE/FALSE needed
